<a href="https://colab.research.google.com/github/GuiDecker/VAR-AI/blob/main/VAR-Vigilancia-Ambiental-Resiliente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google-generativeai
!pip install -U -q ultralytics

In [ ]:
import google.generativeai as genai
from ultralytics import YOLO

# ***Dataset treinado YOLO versão 8.***
  Faça o upload do arquivo .pt (Disponivel no repositório do github)

  EXEMPLO:


```
model = YOLO("/content/yolov8n.pt")

 model.predict(source="/content/areas-de-risco.jpeg", conf=0.7, save=True, show=True)
```

In [ ]:
# EXEMPLO: model = YOLO("/content/yolov8n.pt")
model = YOLO("<ARQUIVO_YOLO_AQUI>")

# Irá fazer a detecção de pessoas, animais, objetos utilizando visão computacional

model.predict(source="<CAMINHO_ARQUIVO_AQUI>", conf=0.7, save=True, show=True)
#EXEMPLO:  model.predict(source="/content/areas-de-risco.jpeg", conf=0.7, save=True, show=True)

# Extração de frames do arquivo

Como o Gemini nâo aceita arquivos de vídeos diretamente, é necessario quebar o vídeo em multiplas frames de imagens de 1 segundo

Você pode colocar um arquivo local ou uma url que contenha algum arquivo de imagem ou vídeo

In [ ]:
import cv2
import os
import shutil

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# O vídeo comentádo de 10min é um vídeo padrão do sherlock homes, caso queira testa-lo também, sugiro passar a variavél full_video como False
# video_file_name = "https://storage.googleapis.com/generativeai-downloads/data/SherlockJr._10min.mp4"

# Vídeo local de desastre ambiental das enchentes no Rio Grande do Sul
video_file_name = "/content/runs/detect/predict/RS_video.avi"


# Crie ou limpe o diretório de frames de imagens extraídos existente.
FRAME_EXTRACTION_DIRECTORY = "/content/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extraindo {video_file_path} a 1 quadro por segundo. pode levar um tempo...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps # Intervalo de tempo entre frames (em segundos)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # Final do video
          break
      if int(count / fps) == frame_count: # Extrai um frame a cada segundo
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release()
  print(f"Extração de frames de vídeo concluída!\n\Extraído: {frame_count} frames")

extract_frame_from_video(video_file_name)

# **Upload de frames usando o File API**

Depois de extrair, faça o upload da captura dos frames tiradas do arquivo da variavel **video_file_name**

File API aceita arquivos abaixo de 2GB e pode armazenar até 20GB de arquivos por projeto.

In [ ]:

import os

class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extrai a contagem de Frames (como um número inteiro) de um nome de arquivo com o formato
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indica que o nome do arquivo pode estar formatado incorretamente
  return parts[1].split('.')[0]

# Processa cada Frame do vídeo no diretório de saída
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

#Enviar os arquivos para a API
# Carregando apenas uma parte teste de 10 segundos de arquivos para reduzir o tempo de upload.
# Altere full_video para True para enviar o vídeo inteiro e False para reduzido.
full_video = True

uploaded_files = []
print(f'Uploading {len(files_to_upload) if full_video else 10} files. Isso pode levar um tempo...')

for file in files_to_upload if full_video else files_to_upload[40:50]:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Uploads de arquivos concluídos!\n\nUploaded: {len(uploaded_files)} files")


# **Lista os arquivos enviados na API\**

Verificação para ver se os arquivos foram publicados com sucesso no File API

In [ ]:
# Lista os arquivos enviados na API
for n, f in zip(range(len(uploaded_files)), genai.list_files()):
  print(f.uri)

# **Geração e analise do conteúdo com genai**

In [ ]:
# Criação prompt.
prompt = "Analise o vídeo cheque se o ambiente se passa em algum desastre, acidente ou confusão. E se tiver acontecido alguma enchente, fogo, terremoto, destruição de coisas, faça um passo a passo preciso de ações especificas para ajudar as pessoas dependendo da condição do ambiente, e também traga ações imeditas dependendo do tipo de pessoa, seja ela criança ou idosa ou animal. E principalmente analise se tem qualquer sinal de pessoas e descreva como e onde essa pessoa estava no vídeo, levantando descrições de ações de resgate para autoridades."

model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Faça a solicitação GenerateContent com a estrutura descrita acima.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Execução da solicitação LLM.
request = make_request(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

# **Deletar Arquivos que publicados na API, caso queira liberar espaço**

In [ ]:
print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
for file in uploaded_files:
  genai.delete_file(file.response.name)
  print(f'Deleted {file.file_path} at URI {file.response.uri}')
print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")